In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
img_size=2000
np.random.seed(1234567)
#loading data and fitting follows:
Xtrain=np.loadtxt('training_data/Xtrain_big.csv',delimiter=',')
Ytrain=np.loadtxt('training_data/YtrainR_big.csv',delimiter=',')
#Xtrain=Xtrain[:2000]
#Xtrain=np.transpose(Xtrain)
print(Xtrain.shape)

In [ ]:
#defining a sampling function: returns a random sample from a mean and variance
#that is input to it
def sampling(mu_log_variance):
    mu, log_variance = mu_log_variance
    epsilon = keras.backend.random_normal(shape=keras.backend.shape(mu), mean=0.0, stddev=1.0)
    random_sample = mu + keras.backend.exp(log_variance/2) * epsilon
    return random_sample

In [ ]:
#model defined here
inp = keras.layers.Input(shape=(img_size, 1), name="encoder_input")
#leaky reLU has been added as a separate layer than define as an activation
x1 = keras.layers.Conv1D(filters=1, kernel_size=5, padding="same", strides=1,name="encoder_conv_1")(inp)
x1 = keras.layers.BatchNormalization(name="encoder_norm_1")(x1)
x1 = keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(x1)

x1 = keras.layers.Conv1D(filters=16, kernel_size=5, padding="same", strides=1, name="encoder_conv_2")(x1)
x1 = keras.layers.BatchNormalization(name="encoder_norm_2")(x1)
x1 = keras.layers.LeakyReLU(name="encoder_leakyrelu_2")(x1)

x1 = keras.layers.Conv1D(filters=32, kernel_size=5, padding="same", strides=1, name="encoder_conv_3")(x1)
x1 = keras.layers.BatchNormalization(name="encoder_norm_3")(x1)
bp_lay_1 = keras.layers.LeakyReLU(name="encoder_leakyrelu_3")(x1)
#flatten the layers in encoder
shape_before_flatten = keras.backend.int_shape(bp_lay_1)[1:]
x2 = keras.layers.Flatten(name="flat_1")(bp_lay_1)

latent_space_dim = 8
#declare a mean and variance for the distribution
encoder_mu = keras.layers.Dense(units=latent_space_dim, name="encoder_mu")(x2)
encoder_log_variance = keras.layers.Dense(units=latent_space_dim, name="encoder_log_variance")(x2)
encoder_op = keras.layers.Lambda(sampling, name="encoder_output")([encoder_mu, encoder_log_variance])

encoder = keras.models.Model(inp, encoder_op, name="encoder_model")
encoder.summary()


In [ ]:
#decoder_input = keras.layers.Input(shape=(shape_before_flatten), name="decoder_input")
decoder_input = keras.layers.Input(shape=(latent_space_dim), name="decoder_input")
x3 = keras.layers.Dense(units=np.prod(shape_before_flatten), name="decoder_dense_1")(decoder_input)
x3 = keras.layers.Reshape(target_shape=shape_before_flatten)(x3)

x3 = keras.layers.Conv1DTranspose(filters=32, kernel_size=5, padding="same", strides=1, name="decoder_conv_tran_1")(x3)
x3 = keras.layers.BatchNormalization(name="decoder_norm_1")(x3)
x3 = keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(x3)

x3 = keras.layers.Conv1DTranspose(filters=16, kernel_size=5, padding="same", strides=1, name="decoder_conv_tran_2")(x3)
x3 = keras.layers.BatchNormalization(name="decoder_norm_2")(x3)
x3 = keras.layers.LeakyReLU(name="decoder_leakyrelu_2")(x3)

decoder_output = keras.layers.Conv1DTranspose(filters=1, kernel_size=5, padding="same", strides=1,activation='sigmoid', name="decoder_conv_tran_4")(x3)
#decoder_output = keras.layers.LeakyReLU(name="decoder_output")(x3)

decoder = keras.models.Model(decoder_input, decoder_output, name="decoder_model")
decoder.summary()


In [ ]:
from keras import backend as K
def loss_func(encoder_mu, encoder_log_variance):
    def vae_reconstruction_loss(y_true, y_predict):
        reconstruction_loss_factor = 1000
        reconstruction_loss = keras.backend.mean(keras.backend.square(y_true-y_predict), axis=[1, 2])
        return reconstruction_loss_factor * reconstruction_loss

    def vae_kl_loss(encoder_mu, encoder_log_variance):
        kl_loss = -0.5 * keras.backend.sum(1.0 + encoder_log_variance - keras.backend.square(encoder_mu) - keras.backend.exp(encoder_log_variance), axis=1)
        return kl_loss

    def vae_kl_loss_metric(y_true, y_predict):
        kl_loss = -0.5 * keras.backend.sum(1.0 + encoder_log_variance - keras.backend.square(encoder_mu) - keras.backend.exp(encoder_log_variance), axis=1)
        return kl_loss

    def vae_loss(y_true, y_predict):
        reconstruction_loss = vae_reconstruction_loss(y_true, y_predict)
        kl_loss = vae_kl_loss(y_true, y_predict)

        loss = reconstruction_loss + kl_loss
        return loss

    return vae_loss



In [ ]:
#model for the vae
vae_input = keras.layers.Input(shape=(img_size,1), name="VAE_input")
vae_encoder_output = encoder(vae_input)
vae_decoder_output = decoder(vae_encoder_output)
vae = keras.models.Model(vae_input, vae_decoder_output, name="VAE")

vae.summary()
#vae.compile(optimizer=keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
vae.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0005), loss=loss_func(encoder_mu, encoder_log_variance))



In [ ]:
#Xtrain_N=[]
#[Xtrain_N.append(Xtrain[i]/np.abs(Xtrain[i][np.argmin(Xtrain[i])])) for i in range(0,3231)]
#Xtrain=np.array(Xtrain_N).reshape(3231,12000,1)
arr=np.arange(0,4500,1)
np.random.shuffle(arr)
Xtrain=np.array([Xtrain[i] for i in arr])
Xtrain=np.array(Xtrain).reshape(4500,2000,1)
Xtrain=Xtrain[0:4500]
print(Xtrain.shape)


In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history=vae.fit(Xtrain, Xtrain, epochs=30, batch_size=64 ,verbose=1, validation_split=0.2)

In [ ]:
vae.save_weights('autoencoder_simpler_moreD.h5')

In [ ]:
encoder.save_weights('encoding_simpler_moreD.h5')
decoder.save_weights('decoding_simpler_moreD.h5')